Dependencies

In [1]:
import subprocess
import json
import re
import os
import cma
import csv
import pandas as pd
import numpy as np
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
import pymoo
print(pymoo.__version__)
import matplotlib.pyplot as plt
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.util.ref_dirs import get_reference_directions

0.6.1.3


Datasets

In [2]:
datasets = {
    "xor": {
        "train": "./dat/train_xor.dat",
        "test": "./dat/test_xor.dat"
    },
    "compas": {
        "train": "./dat/train_compas.dat",
        "test": "./dat/test_compas.dat"
    },
    "nomnist": {
        "train": "./dat/train_nomnist.dat",
        "test": "./dat/test_nomnist.dat"
    }
}

Hiperparameters values

In [3]:
# Valores discretos para las capas y las neuronas
if True:    
    layers_range = [1, 2, 3, 4]
    neurons_options = [2, 4, 8, 16, 32, 64, 128]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.0001, 0.7)
    mu_range = (0.0001, 1.0)
else:
    layers_range = [1]
    neurons_options = [2, 4]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.01, 0.1)
    mu_range = (0.01, 0.1)

In [4]:
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\nTest error \(Mean \+- SD\):\s+([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\s*Test\s+error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"
train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"

CMA-ES ALGORTIHM OPTIMIZATION - LOOKING FOR THE BEST SET OF PARAMETERS

In [5]:
def run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function):
    # Construir el comando con las nuevas flags
    if train_file != "train_nomnist.dat":
        iterations = "1000"
    else:
        iterations = "500"
    command = [
        "./bin/la2", "-t", train_file, "-T", test_file,
        "-e", str(eta), "-m", str(mu), "-l", str(layers), "-h", str(neurons), "-i", iterations,
        "-f", str(error_function)
    ]
    # Añadir el modo offline si corresponde
    if offline == 1:
        command.append("-o")

    # Ejecutar el programa
    result = subprocess.run(command, capture_output=True, text=True)

    # Buscar los valores de error
    match = re.search(train_test_regex, result.stdout, re.DOTALL)
    if match:
        train_mean, train_std, test_mean, test_std = match.groups()
        return {
            "train_mean": float(train_mean),
            "train_std": float(train_std),
            "test_mean": float(test_mean),
            "test_std": float(test_std)
        }
    return None

In [6]:
def evaluate_fitness(params, train_file, test_file):
    # Limitar eta y mu dentro de sus rangos permitidos
    eta = max(min(params[0], eta_range[1]), eta_range[0])
    mu = max(min(params[1], mu_range[1]), mu_range[0])

    # Limitar otros parámetros como antes
    layers = int(max(min(params[2], max(layers_range)), min(layers_range)))
    neurons_index = int(max(min(params[3], len(neurons_options) - 1), 0))
    neurons = neurons_options[neurons_index]
    offline = int(max(min(params[4], 1), 0))
    error_function = int(max(min(params[5], 1), 0))

    # Ejecutar el programa y obtener resultados
    result = run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function)

    if result:
        return result["test_mean"], result  # Devuelve el test_mean y el resto del resultado
    return float("inf"), None  # Penalizar si hay fallos y no hay resultado


In [7]:
def save_to_csv(filename, results):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Encabezado del CSV
        writer.writerow(["Eta", "Mu", "Layers", "Neurons", "Offline", "Error Function",
                         "Train Mean", "Train Std", "Test Mean", "Test Std"])
        # Escribir cada fila de resultados
        for row in results:
            writer.writerow(row)

In [8]:
# Bucle principal actualizado
def optimize_with_cma_es(dataset_key):
    if dataset_key not in datasets:
        print(f"Dataset '{dataset_key}' no encontrado.")
        return

    # Archivos de entrenamiento y prueba
    train_file = datasets[dataset_key]["train"]
    test_file = datasets[dataset_key]["test"]

    # Configuración inicial de CMA-ES
    initial_params = [0.01, 0.5, 2, 3, 1, 0]  # [eta, mu, layers, neurons_index, offline, error_function]

    sigma = 0.1 # Rango de búsqueda inicial
    es = cma.CMAEvolutionStrategy(
    initial_params,
    sigma,
    {
        'bounds': [
            [eta_range[0], mu_range[0], 1, 0, 0, 0],  # Límite inferior
            [eta_range[1], mu_range[1], max(layers_range), len(neurons_options) - 1, 1, 1]  # Límite superior
        ]
    }
)

    # Resultados almacenados
    results = []

    # Bucle de optimización
    while not es.stop():
        solutions = es.ask()
        fitness = []
        for solution in solutions:
            # Evaluar cada solución
            test_mean, result = evaluate_fitness(solution, train_file, test_file)
            fitness.append(test_mean)  # Usar solo test_mean como fitness

            if result:  # Guardar resultados solo si son válidos
                eta, mu = solution[0], solution[1]
                layers = int(max(min(solution[2], max(layers_range)), min(layers_range)))
                neurons_index = int(max(min(solution[3], len(neurons_options) - 1), 0))
                neurons = neurons_options[neurons_index]
                offline = int(max(min(solution[4], 1), 0))
                error_function = int(max(min(solution[5], 1), 0))

                # Almacenar resultados
                results.append([
                    eta, mu, layers, neurons, offline, error_function,
                    result["train_mean"], result["train_std"], result["test_mean"], result["test_std"]
                ])

        # Informar a CMA-ES
        es.tell(solutions, fitness)
        print(f"Generación {es.result.iterations}: Mejor error de prueba = {min(fitness)}")

    return results

In [9]:
resultados = []

In [10]:
nombre = "xor"
optimize_with_cma_es(nombre)
# resultados.append(pd.read_csv(f"results_{nombre}.csv"))

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=1106380, Mon Dec 23 22:00:14 2024)
Generación 1: Mejor error de prueba = 0.000430869
Generación 2: Mejor error de prueba = 0.000335559
Generación 3: Mejor error de prueba = 0.000248592
Generación 4: Mejor error de prueba = 0.000192691
Generación 5: Mejor error de prueba = 0.000141586
Generación 6: Mejor error de prueba = 0.000144918
Generación 7: Mejor error de prueba = 0.000143165
Generación 8: Mejor error de prueba = 0.000133056
Generación 9: Mejor error de prueba = 0.000132675
Generación 10: Mejor error de prueba = 0.000133903
Generación 11: Mejor error de prueba = 0.000134533
Generación 12: Mejor error de prueba = 0.000134357
Generación 13: Mejor error de prueba = 0.000107661
Generación 14: Mejor error de prueba = 0.00011659
Generación 15: Mejor error de prueba = 9.37862e-05
Generación 16: Mejor error de prueba = 8.61579e-05
Generación 17: Mejor error de prueba = 9.20483e-05
Generación 18: Mejor error de prueba = 9.25568e-05
G

KeyboardInterrupt: 

In [30]:
#print(resultados[0].sort_values(by=["Test Mean", "Train Mean", "Layers", "Neurons"], ascending=True).head(10))

MULTI-OBJETIVE OPTIMIZATION: FOR THE CHEAPEST SET OF PARAMETERS

In [11]:
import numpy as np
import random
import subprocess
import re
from concurrent.futures import ThreadPoolExecutor

# Definir parámetros de optimización
LAYER_OPTIONS = [1, 2, 3, 4]  # Valores discretos
NEURON_OPTIONS = [2, 4, 8, 16, 32, 64, 128]  # Valores discretos
ETA_RANGE = (0.0001, 0.9)  # Continuo
MU_RANGE = (0.0001, 0.9)  # Continuo
POP_SIZE = 50
GENERATIONS = 100
# Expresión regular para capturar errores de entrenamiento y prueba
train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"


# Función para generar un individuo
def generate_individual():
    layers = random.choice(LAYER_OPTIONS)
    neurons = random.choice(NEURON_OPTIONS)
    eta = random.uniform(*ETA_RANGE)
    mu = random.uniform(*MU_RANGE)
    offline = random.choice([0, 1])  # Discreto
    error_function = random.choice([0, 1])  # Discreto
    return [layers, neurons, eta, mu, offline, error_function]

# Función para inicializar la población
def initialize_population(size):
    return [generate_individual() for _ in range(size)]

# Evaluar la población en paralelo
def evaluate_population(population, dataset_key):
    dataset_files = datasets[dataset_key]
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda ind: evaluate(ind, dataset_files), population))
    return results

# Función de evaluación
def evaluate(individual, dataset_files):
    train_file = dataset_files["train"]
    test_file = dataset_files["test"]

    if train_file != "./dat/train_nomnist.dat":
        iterations = "1000"
    else:
        iterations = "500"
    
    layers, neurons, eta, mu, offline, error_function = individual

    # Construir el comando para ejecutar la red neuronal
    command = [
        "./bin/la2",
        "-l", str(layers),
        "-h", str(neurons),
        "-e", str(eta),
        "-m", str(mu),
        "-o", str(offline),
        "-f", str(error_function),
        "-t", train_file,
        "-T", test_file,
        "-i", iterations
    ]

    # Ejecutar el comando y capturar la salida
    result = subprocess.run(command, capture_output=True, text=True)

    # Extraer los errores usando la regex
    match = re.search(train_test_regex, result.stdout)
    if match:
        train_error = float(match.group(1))
        test_error = float(match.group(3))
    else:
        # Penalizar si no se puede ejecutar correctamente
        train_error = float("inf")
        test_error = float("inf")

    # Calcular el número total de neuronas
    total_neurons = layers * neurons

    # Retornar las funciones objetivo
    return train_error, test_error, total_neurons

# Función para calcular si un individuo domina a otro
def dominates(ind1, ind2):
    return all(x <= y for x, y in zip(ind1, ind2)) and any(x < y for x, y in zip(ind1, ind2))

# Función para actualizar el frente de Pareto
def update_pareto_front(front, individual, objectives):
    to_remove = []
    dominated = False

    for i, (ind, obj) in enumerate(front):
        if dominates(objectives, obj):
            to_remove.append(i)
        elif dominates(obj, objectives):
            dominated = True

    for i in sorted(to_remove, reverse=True):
        del front[i]

    if not dominated:
        front.append((individual, objectives))

# Selección por torneo binario
def tournament_selection(population, objectives):
    i, j = random.sample(range(len(population)), 2)
    if dominates(objectives[i], objectives[j]):
        return population[i]
    elif dominates(objectives[j], objectives[i]):
        return population[j]
    return random.choice([population[i], population[j]])

# Cruce uniforme
def crossover(parent1, parent2):
    return [
        (p1 + p2) / 2 if isinstance(p1, float) else random.choice([p1, p2])
        for p1, p2 in zip(parent1, parent2)
    ]

# Mutación
def mutate(individual, mutation_rate=0.2):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            if i in [0, 1]:  # Discretos
                individual[i] = random.choice(LAYER_OPTIONS if i == 0 else NEURON_OPTIONS)
            elif i in [2, 3]:  # Continuos
                range_val = ETA_RANGE if i == 2 else MU_RANGE
                individual[i] = random.uniform(*range_val)
            elif i in [4, 5]:  # Binarios
                individual[i] = 1 - individual[i]

# Algoritmo evolutivo
def evolutionary_algorithm(dataset_key):
    population = initialize_population(POP_SIZE)
    pareto_front = []

    for gen in range(GENERATIONS):
        new_population = []
        objectives = evaluate_population(population, dataset_key)

        for ind, obj in zip(population, objectives):
            update_pareto_front(pareto_front, ind, obj)

        for _ in range(POP_SIZE):
            parent1 = tournament_selection(population, objectives)
            parent2 = tournament_selection(population, objectives)
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)

        population = new_population
        print(f"Generación {gen + 1}: Frente de Pareto tiene {len(pareto_front)} soluciones")

    return pareto_front

In [ ]:
import os
import re
import csv
import subprocess
import matplotlib.pyplot as plt
from collections import defaultdict

# Diccionario de datasets
datasets = {
    "xor": {
        "train": "./dat/train_xor.dat",
        "test": "./dat/test_xor.dat"
    },
    "compas": {
        "train": "./dat/train_compas.dat",
        "test": "./dat/test_compas.dat"
    },
    "nomnist": {
        "train": "./dat/train_nomnist.dat",
        "test": "./dat/test_nomnist.dat"
    }
}

# Crear carpeta de resultados si no existe
if not os.path.exists("resultados"):
    os.makedirs("resultados")

# Función para transformar resultados de Pareto al formato de CMA-ES
def transform_pareto_results(pareto_results):
    transformed = []
    for ind, obj in pareto_results:
        layers, neurons, eta, mu, offline, error_function = ind
        train_error, test_error, total_neurons = obj
        transformed.append([
            eta, mu, layers, neurons, offline, error_function,
            train_error, 0.0,  # train_std (no disponible en Pareto)
            test_error, 0.0   # test_std (no disponible en Pareto)
        ])
    return transformed

# Función para guardar resultados en CSV
def save_results_to_csv(results, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["eta", "mu", "layers", "neurons", "offline", "error_function", 
                         "train_mean", "train_std", "test_mean", "test_std"])
        writer.writerows(results)

# Función para guardar resultados de configuraciones top en archivos .txt
def save_top_configurations(results, filename, top_n=5):
    sorted_results = sorted(results, key=lambda x: x[8])  # Ordenar por test_mean (columna 8)
    with open(filename, mode='w') as file:
        for rank, config in enumerate(sorted_results[:top_n], start=1):
            file.write(f"Top {rank}:\n")
            file.write(f"eta: {config[0]}, mu: {config[1]}, layers: {config[2]}, neurons: {config[3]}, "
                       f"offline: {config[4]}, error_function: {config[5]}, "
                       f"train_mean: {config[6]}, train_std: {config[7]}, "
                       f"test_mean: {config[8]}, test_std: {config[9]}\n\n")

# Función para ejecutar las configuraciones top y guardar los resultados en .txt
def execute_top_configurations(top_configs, dataset_key, filename):
    train_file = datasets[dataset_key]["train"]
    test_file = datasets[dataset_key]["test"]
    with open(filename, mode='w') as file:
        for i, config in enumerate(top_configs):
            command = [
                "./bin/la2",
                "-l", str(config[2]), "-h", str(config[3]),
                "-e", str(config[0]), "-m", str(config[1]),
                "-o", str(config[4]), "-f", str(config[5]),
                "-t", train_file, "-T", test_file, "-i", "1000"
            ]
            result = subprocess.run(command, capture_output=True, text=True)
            file.write(f"Top {i + 1}:\n")
            file.write(result.stdout + "\n")

# Función para leer vectores de resultados desde los archivos .txt
def leer_vectores():
    seed_pattern = re.compile(r"SEED (\d+)")
    iteration_pattern = re.compile(r"Iteration (\d+)\s+Training error:\s+([\d.]+)\s+Test error:\s+([\d.]+)")
    results = defaultdict(lambda: defaultdict(list))

    for filename in os.listdir("resultados"):
        if filename.startswith("top_") and filename.endswith(".txt"):
            filepath = os.path.join("resultados", filename)
            with open(filepath) as file:
                current_seed = None
                for line in file:
                    seed_match = seed_pattern.search(line)
                    if seed_match:
                        current_seed = int(seed_match.group(1))
                        continue
                    iteration_match = iteration_pattern.search(line)
                    if iteration_match and current_seed is not None:
                        iteration = int(iteration_match.group(1))
                        training_error = float(iteration_match.group(2))
                        test_error = float(iteration_match.group(3))
                        results[filename]["iterations"].append(iteration)
                        results[filename]["training_errors"].append(training_error)
                        results[filename]["test_errors"].append(test_error)
    return results

# Función para generar gráficas de convergencia
def plot_convergence(results):
    for filename, data in results.items():
        iterations = data["iterations"]
        training_errors = data["training_errors"]
        test_errors = data["test_errors"]
        
        plt.figure()
        plt.plot(iterations, training_errors, label="Training Error", color="orange")
        plt.plot(iterations, test_errors, label="Test Error", color="blue")
        plt.xlabel("Iterations")
        plt.ylabel("Error")
        plt.title(f"Convergence Plot: {filename}")
        plt.legend()
        plt.savefig(os.path.join("resultados", f"{filename}_convergence.png"))
        plt.close()

# Automatizar el estudio
def run_study():
    for dataset_key in datasets.keys():
        # Ejecutar CMA-ES
        if dataset_key != "xor" or dataset_key != "compas":
            print(f"Ejecutando CMA-ES para {dataset_key}")
            cma_results = optimize_with_cma_es(dataset_key)
            cma_filename = f"resultados/resultados_cma_{dataset_key}.csv"
            save_results_to_csv(cma_results, cma_filename)
            save_top_configurations(cma_results, f"resultados/top_cma_{dataset_key}.txt")

        # Ejecutar algoritmo evolutivo
        if dataset_key != "xor":
            print(f"Ejecutando algoritmo evolutivo para {dataset_key}")
            pareto_results = evolutionary_algorithm(dataset_key)
            pareto_transformed = transform_pareto_results(pareto_results)
            pareto_filename = f"resultados/resultados_pareto_{dataset_key}.csv"
            save_results_to_csv(pareto_transformed, pareto_filename)
            save_top_configurations(pareto_transformed, f"resultados/top_pareto_{dataset_key}.txt")

        # Ejecutar las configuraciones top
        print(f"Ejecutando configuraciones top para {dataset_key}")
        top_cma_configs = sorted(cma_results, key=lambda x: x[8])[:5]
        execute_top_configurations(top_cma_configs, dataset_key, f"resultados/top_5_cma_{dataset_key}.txt")

        top_pareto_configs = sorted(pareto_transformed, key=lambda x: x[8])[:5]  # Ordenar por test_mean
        execute_top_configurations(top_pareto_configs, dataset_key, f"resultados/top_5_pareto_{dataset_key}.txt")

    # Leer vectores y graficar
    print("Generando gráficas de convergencia")
    all_results = leer_vectores()
    plot_convergence(all_results)

# Ejecutar el estudio completo
run_study()


Ejecutando CMA-ES para xor
(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=1116411, Mon Dec 23 22:02:45 2024)
Generación 1: Mejor error de prueba = 0.000403771
Generación 2: Mejor error de prueba = 0.000463631
Generación 3: Mejor error de prueba = 0.000300716
Generación 4: Mejor error de prueba = 0.000291645
Generación 5: Mejor error de prueba = 0.000227401
Generación 6: Mejor error de prueba = 0.000222407
Generación 7: Mejor error de prueba = 0.00017767
Generación 8: Mejor error de prueba = 7.41936e-05
Generación 9: Mejor error de prueba = 7.44257e-05
Generación 10: Mejor error de prueba = 8.13477e-05
Generación 11: Mejor error de prueba = 8.60286e-05
Generación 12: Mejor error de prueba = 8.52412e-05
Generación 13: Mejor error de prueba = 8.28908e-05
Generación 14: Mejor error de prueba = 7.98422e-05
Generación 15: Mejor error de prueba = 7.17111e-05
Generación 16: Mejor error de prueba = 7.54044e-05
Generación 17: Mejor error de prueba = 7.65417e-05
Generación 18: Mejor erro

In [33]:
import os

def run_study():
    datasets_keys = ["xor", "compas", "nomnist"]
    informe_lines = []

    for dataset_key in datasets_keys:
        print(f"Iniciando estudio para {dataset_key}...")

        # Ejecutar y guardar los mejores 5 resultados de CMA-ES
        cma_results = optimize_with_cma_es(dataset_key)
        sorted_cma_results = sorted(cma_results, key=lambda x: x[-1])[:5]  # Ordenar por test error
        for i, result in enumerate(sorted_cma_results):
            filename = f"top_{i + 1}_cma_{dataset_key}.txt"
            with open(os.path.join("resultados", filename), "w") as f:
                f.write(f"# Resultados de CMA-ES para {dataset_key}\n")
                f.write("eta, mu, layers, neurons, offline, error_function, train_error, train_std, test_error, test_std\n")
                f.write(", ".join(map(str, result)) + "\n")
            informe_lines.append(f"{filename}: {result}")

        # Ejecutar y guardar los mejores 5 resultados del Pareto
        pareto_results = evolutionary_algorithm(dataset_key)
        sorted_pareto_results = sorted(pareto_results, key=lambda x: x[1][1])[:5]  # Ordenar por test error
        for i, (ind, obj) in enumerate(sorted_pareto_results):
            filename = f"top_{i + 1}_pareto_{dataset_key}.txt"
            with open(os.path.join("resultados", filename), "w") as f:
                f.write(f"# Resultados de Pareto para {dataset_key}\n")
                f.write("layers, neurons, eta, mu, offline, error_function, train_error, test_error, total_neurons\n")
                f.write(", ".join(map(str, ind)) + "\n")
                f.write(", ".join(map(str, obj)) + "\n")
            informe_lines.append(f"{filename}: {ind}, {obj}")

    # Guardar el archivo informe
    with open(os.path.join("resultados", "informe.txt"), "w") as f:
        f.write("# Informe de estudio\n")
        f.write("\n".join(informe_lines))
    print("Estudio completo. Resultados guardados en la carpeta 'resultados'.")

def plot_convergence(results):
    for filename, data in results.items():
        # Parsear los parámetros desde el nombre del archivo
        match = re.search(r"top_\d+_(cma|pareto)_(\w+)\.txt", filename)
        if not match:
            print(f"No se pudo parsear el archivo: {filename}")
            continue

        algorithm = match.group(1)
        dataset = match.group(2)

        # Obtener iteraciones, errores
        iterations = data["iterations"]
        training_errors = data["training_errors"]
        test_errors = data["test_errors"]

        # Generar nombres descriptivos de salida
        output_path = os.path.join("resultados", f"{filename.replace('.txt', '.png')}")

        # Graficar los errores
        plt.figure()
        plt.plot(iterations, training_errors, label="Training Error", color="orange")
        plt.plot(iterations, test_errors, label="Test Error", color="blue")
        plt.xlabel("Iterations")
        plt.ylabel("Error")
        plt.title(f"Convergence Plot for {dataset} ({algorithm})")
        plt.legend()
        plt.savefig(output_path)
        plt.close()

        print(f"Gráfica guardada: {output_path}")


In [34]:
run_study()
plot_convergence(leer_vectores())

Iniciando estudio para xor...


ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.